In [4]:
import pandas as pd
import shutil
import os

In [5]:
# ============================
# 1. Label 수정
# ============================
train_df = pd.read_csv('train.csv')

# 전 기수가 발견한 잘못된 label 수정
corrections = {
    "45f0d2dfc7e47c03.jpg": 7,
    "aec62dced7af97cd.jpg": 14,
    "0583254a73b48ece.jpg": 10,
    "1ec14a14bbe633db.jpg": 7,
    "c5182ab809478f12.jpg": 14,
    "8646f2c3280a4f49.jpg": 3,
    "38d1796b6ad99ddd.jpg": 10,
}

print("Label 수정:")
for file, correct_target in corrections.items():
    if file in train_df['ID'].values:
        old_target = train_df[train_df['ID'] == file]['target'].values[0]
        train_df.loc[train_df['ID'] == file, 'target'] = correct_target
        print(f"  ✅ {file}: {old_target} → {correct_target}")

# 수정된 CSV 저장
train_df.to_csv('train_corrected.csv', index=False)
print(f"\n✅ 수정된 train.csv 저장 완료: train_corrected.csv")

# ============================
# 수정 사항 검증
# ============================
print("\n" + "="*70)
print("🔍 수정 사항 검증")
print("="*70)

# 저장된 파일 다시 읽어서 확인
verify_df = pd.read_csv('train_corrected.csv')
all_correct = True

for file, correct_target in corrections.items():
    row = verify_df[verify_df['ID'] == file]
    if len(row) > 0:
        actual_target = row['target'].values[0]
        if actual_target == correct_target:
            print(f"  ✅ {file}: 타겟 = {actual_target} (올바름)")
        else:
            print(f"  ❌ {file}: 타겟 = {actual_target} (기대값: {correct_target})")
            all_correct = False
    else:
        print(f"  ⚠️  {file}: 파일에서 찾을 수 없음")
        all_correct = False

print("="*70)
if all_correct:
    print("🎉 모든 레이블이 올바르게 수정되고 저장되었습니다!")
else:
    print("⚠️  일부 레이블 수정에 문제가 있습니다. 다시 확인하세요.")
print("="*70)


Label 수정:
  ✅ 45f0d2dfc7e47c03.jpg: 3 → 7
  ✅ aec62dced7af97cd.jpg: 3 → 14
  ✅ 0583254a73b48ece.jpg: 11 → 10
  ✅ 1ec14a14bbe633db.jpg: 14 → 7
  ✅ c5182ab809478f12.jpg: 4 → 14
  ✅ 8646f2c3280a4f49.jpg: 7 → 3
  ✅ 38d1796b6ad99ddd.jpg: 11 → 10

✅ 수정된 train.csv 저장 완료: train_corrected.csv

🔍 수정 사항 검증
  ✅ 45f0d2dfc7e47c03.jpg: 타겟 = 7 (올바름)
  ✅ aec62dced7af97cd.jpg: 타겟 = 14 (올바름)
  ✅ 0583254a73b48ece.jpg: 타겟 = 10 (올바름)
  ✅ 1ec14a14bbe633db.jpg: 타겟 = 7 (올바름)
  ✅ c5182ab809478f12.jpg: 타겟 = 14 (올바름)
  ✅ 8646f2c3280a4f49.jpg: 타겟 = 3 (올바름)
  ✅ 38d1796b6ad99ddd.jpg: 타겟 = 10 (올바름)
🎉 모든 레이블이 올바르게 수정되고 저장되었습니다!


In [6]:
# meta.csv 불러오기 및 클래스 리스트 생성
meta_df = pd.read_csv('meta.csv')
classes = list(meta_df['class_name'].unique())


# 수정된 CSV 저장
train_df.to_csv('train_corrected.csv', index=False)
print(f"\n✅ 수정된 train.csv 저장 완료: train_corrected.csv")

# 혼동 클래스 필터링 및 저장
confused_classes = [1, 13, 14]

confused_df = train_df[train_df['target'].isin(confused_classes)].reset_index(drop=True)
confused_df.to_csv('train_confused_classes.csv', index=False)

# 혼동 클래스만 복사 (옵션)
os.makedirs('train_confused_by_class', exist_ok=True)
for idx, row in confused_df.iterrows():
    img_id = row['ID']
    target = row['target']
    class_name = classes[target]

    class_dir = f'train_confused_by_class/{target}_{class_name}'
    os.makedirs(class_dir, exist_ok=True)

    src = f'train/{img_id}'
    dst = f'{class_dir}/{img_id}'

    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copy(src, dst)

print(f"\n✅ 혼동 클래스 샘플만 복사 완료: train_confused_by_class/")


✅ 수정된 train.csv 저장 완료: train_corrected.csv

✅ 혼동 클래스 샘플만 복사 완료: train_confused_by_class/


In [7]:
# ============================
# 2. 클래스별 이미지 저장 (선택사항)
# ============================
meta_df = pd.read_csv('meta.csv')
classes = list(meta_df['class_name'].unique())

os.makedirs('train_by_class', exist_ok=True)

for idx, row in train_df.iterrows():
    img_id = row['ID']
    target = row['target']
    class_name = classes[target]
    
    class_dir = f'train_by_class/{target}_{class_name}'
    os.makedirs(class_dir, exist_ok=True)
    
    src = f'train/{img_id}'
    dst = f'{class_dir}/{img_id}'
    
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copy(src, dst)

print(f"\n✅ 클래스별 이미지 저장 완료: train_by_class/")


✅ 클래스별 이미지 저장 완료: train_by_class/


In [8]:
# ============================
# 3. Train/Val Split (Stratified)
# ============================
from sklearn.model_selection import train_test_split

train, val = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=42, 
    stratify=train_df['target']
)

train.to_csv('train_split.csv', index=False)
val.to_csv('val_split.csv', index=False)

print(f"\n✅ Train/Val Split 완료:")
print(f"  - Train: {len(train)}개")
print(f"  - Val: {len(val)}개")


✅ Train/Val Split 완료:
  - Train: 1256개
  - Val: 314개
